<a href="https://colab.research.google.com/github/goya5858/commonlitreadabilityprize/blob/main/working/bart_large_mnli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libs

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
compe_name = 'commonlitreadabilityprize'
%cd /content/drive/MyDrive/kaggle/works/$compe_name/working/

!pip install -q -q -q -U albumentations
!pip install -q -q -q -U torch
!pip install -q -q -q timm
!pip install -q -q -q pytorch_lightning
!pip install -q -q -q -U transformers
!pip install -q -q -q -U sentencepiece

import os
import re
import gc
import sys
import time
import copy
import warnings
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import PIL.Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations
from albumentations.pytorch.transforms import ToTensorV2

import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import CosineAnnealingLR

import timm

import nltk
from wordcloud import WordCloud
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob,Word
from collections import Counter
import string
from torch.nn.utils.rnn import pad_sequence

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import model_selection

import transformers
from transformers import get_linear_schedule_with_warmup, AdamW

from transformers import *

/content/drive/MyDrive/kaggle/works/commonlitreadabilityprize/working
     |████████████████████████████████| 81kB 6.7MB/s 
     |████████████████████████████████| 38.2MB 1.2MB/s 
     |████████████████████████████████| 952kB 41.7MB/s 
     |████████████████████████████████| 296kB 14.2MB/s 
     |████████████████████████████████| 808kB 14.2MB/s 
     |████████████████████████████████| 112kB 56.2MB/s 
     |████████████████████████████████| 276kB 28.6MB/s 
     |████████████████████████████████| 645kB 59.8MB/s 
     |████████████████████████████████| 829kB 54.1MB/s 
     |████████████████████████████████| 1.3MB 61.5MB/s 
     |████████████████████████████████| 296kB 56.4MB/s 
     |████████████████████████████████| 143kB 52.2MB/s 
     |████████████████████████████████| 2.1MB 15.4MB/s 
     |████████████████████████████████| 901kB 61.7MB/s 
     |████████████████████████████████| 3.3MB 59.0MB/s 
     |████████████████████████████████| 1.2MB 14.6MB/s 
[nltk_data] Downloading package stop

# Dataset

In [3]:
DEVICE = 'cuda:0'

#get_tokenizer = RobertaTokenizer
#get_model     = BartForSequenceClassification
get_tokenizer = AutoTokenizer #RobertaTokenizer より汎用性ある気がする　使い勝手的な面で
get_model     = AutoModel

VOCAB_PATH = 'roberta-base'
MODEL_PATH = 'facebook/bart-large-mnli'
#VOCAB_PATH = 'bert-base-uncased'
#MODEL_PATH = 'bert-base-uncased'

In [4]:
ROOT = f"../input/{compe_name}/"

df = pd.read_csv(ROOT+'train.csv')
df.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [5]:
test_df = pd.read_csv(ROOT+'test.csv')
test_df.head()

,id,url_legal,license,excerpt
0,c0f722661,NaN,NaN,My hope lay in Jack's promise that he would ke...
1,f0953f0a5,NaN,NaN,Dotty continued to go to Mrs. Gray's every nig...
2,0df072751,NaN,NaN,It was a bright and cheerful scene that greete...
3,04caf4e0c,https://en.wikipedia.org/wiki/Cell_division,CC BY-SA 3.0,Cell division is the process by which a parent...
4,0e63f8bea,https://en.wikipedia.org/wiki/Debugging,CC BY-SA 3.0,Debugging is the process of finding and resolv...


In [6]:
def prep_text(text_df):
  text_df = text_df.str.replace("\n","",regex=False)
  return text_df.str.replace("\'s",r"s",regex=True).values

df['excerpt']      = prep_text(df['excerpt'])
test_df['excerpt'] = prep_text(test_df['excerpt'])

MAX_SEQUENCE_LENGTH = df['excerpt'].apply(lambda x: len(x.split())).max()

tokenizer = get_tokenizer.from_pretrained(VOCAB_PATH,
                                          model_max_length=MAX_SEQUENCE_LENGTH
                                          )
df['token']        = df['excerpt'].apply(tokenizer)
test_df['token']   = test_df['excerpt'].apply(tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (222 > 205). Running this sequence through the model will result in indexing errors


In [7]:
# trainデータを、targetの値をビニングした値を元に層化fold
def create_folds(data, num_splits):
    # we create a new column called kfold and fill it with -1
    data["kfold"] = -1
    # the next step is to randomize the rows of the data
    data = data.sample(frac=1).reset_index(drop=True)
    # calculate number of bins by Sturge's rule
    # I take the floor of the value, you can also
    # just round it
    num_bins = int(np.floor(1 + np.log2(len(data))))
    # bin targets
    data.loc[:, "bins"] = pd.cut(
        data["target"], bins=num_bins, labels=False
    )
    
    # initiate the kfold class from model_selection module
    kf = model_selection.StratifiedKFold(n_splits=num_splits)
    # fill the new kfold column
    # note that, instead of targets, we use bins!
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    # drop the bins column
    data = data.drop("bins", axis=1)

    # return dataframe with folds
    return data

In [8]:
df = create_folds(df, num_splits=5)

In [9]:
class CLPDataset(Dataset):
  def __init__(self, df):
    super().__init__()
    self.token  = df.token
    self.labels = df.target

  def __len__(self):
    return self.labels.shape[0]
  
  def __getitem__(self, idx):
    if len(self.token.iloc[0]) == 2:
      return (
              torch.tensor(self.token.iloc[idx].input_ids), \
              #torch.tensor(self.token.iloc[idx].token_type_ids), \
              torch.tensor(self.token.iloc[idx].attention_mask)
             ), \
              torch.tensor(self.labels.iloc[idx])
    if len(self.token.iloc[idx]) == 3:
      return (
              torch.tensor(self.token.iloc[idx].input_ids), \
              torch.tensor(self.token.iloc[idx].token_type_ids), \
              torch.tensor(self.token.iloc[idx].attention_mask)
             ), \
              torch.tensor(self.labels.iloc[idx])

In [10]:
def collate_fn(batch):
  inputs, labels = zip(*batch)
  try:
    ids, types, masks = zip(*inputs)
    ids   = pad_sequence(ids, batch_first=True).to(DEVICE)
    types = pad_sequence(types, batch_first=True).to(DEVICE)
    masks = pad_sequence(masks, batch_first=True).to(DEVICE)
    labels= torch.tensor(labels, dtype=torch.float).to(DEVICE)
    return {
        "input_ids"      : ids, \
        "token_type_ids" : types, \
        "attention_mask" : masks
        }, \
        labels
  except ValueError:
    ids, masks = zip(*inputs)
    ids   = pad_sequence(ids, batch_first=True).to(DEVICE)
    #types = pad_sequence(types, batch_first=True).to(DEVICE)
    masks = pad_sequence(masks, batch_first=True).to(DEVICE)
    labels= torch.tensor(labels, dtype=torch.float).to(DEVICE)
    return {
        "input_ids"      : ids, \
        #"token_type_ids" : types, \
        "attention_mask" : masks
        }, \
        labels

In [11]:
# BERT Config
BATCH_SIZE = 4
EPOCHS = 15
NUM_TRAIN_STEPS = int((df.shape[0]/BATCH_SIZE)*EPOCHS)
NUM_WARMUP_STEPS = 0
FOLDS = df.kfold.unique()
NUM_FOLDS = df.kfold.nunique() 

MAX_WORDS = df["excerpt"].apply(lambda x: len(x.split())).max()

In [12]:
def get_dataloaders(df, n_fold, BATCH_SIZE=BATCH_SIZE):
  train_df = df[df['kfold']!=n_fold]
  valid_df = df[df['kfold']==n_fold]

  train_dataset = CLPDataset(df=train_df)
  valid_dataset = CLPDataset(df=valid_df)

  train_loader = DataLoader(
      dataset = train_dataset,
      batch_size = BATCH_SIZE,
      shuffle = True,
      collate_fn=collate_fn,
      #num_workers = -1
    )
  valid_loader = DataLoader(
      dataset = valid_dataset,
      batch_size = BATCH_SIZE,
      shuffle = False,
      collate_fn=collate_fn,
      #num_workers = -1
    )
  return train_loader, valid_loader

# model

In [13]:
_, sample_loader = get_dataloaders(df, 0, 1)
sample_data, _ = iter(sample_loader).next()
sample_data

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1]], device='cuda:0'),
 'input_ids': tensor([[    0,  4688,  7468,     8,    69,  1928,    12, 28638,   439,     7,
           3068,    19,    49, 13

In [14]:
class CLPmodel(nn.Module):
  def __init__(self, check_size=False):
    super().__init__()
    self.model  = get_model.from_pretrained(MODEL_PATH).to(DEVICE)
    OUTPUT_SIZE = self.model(**sample_data)[0].shape[-1]
    self.drop = nn.Dropout(0.5)
    self.fc = nn.Linear(in_features=OUTPUT_SIZE, out_features=1)
    if check_size:
      print('base_model`s output_size :', OUTPUT_SIZE)
      print(DEVICE)
  
  def forward(self,inputs):
    out = self.model(**inputs)
    last_hiddens = out[0]
    out = self.drop(last_hiddens[:,0,:].squeeze(1))
    return self.fc(out)

In [15]:
model = CLPmodel(check_size=True)
del model, _
gc.collect()


base_model`s output_size : 1024
cuda:0


1053

# Training

In [16]:
def train_fn(model, DATA, loss_fn, optim, scheduler):
  optim.zero_grad()
  inputs, labels = DATA[0], DATA[1]
  model = model.to(DEVICE)
  pred = model(inputs)
  loss = loss_fn(pred, labels)
  loss.backward()
  optim.step()
  scheduler.step()
  return loss.detach().cpu().numpy()

def valid_fn(model, DATA, loss_fn, optim):
  with torch.no_grad():
    #optim.zero_grad()
    inputs, labels = DATA[0], DATA[1]
    model = model.to(DEVICE)
    pred = model(inputs)
    loss = loss_fn(pred, labels)
    #loss.backward()
    #optim.step()
  return loss.detach().cpu().numpy(), pred.cpu().detach().numpy()

In [17]:
# RMSE
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.eps = 1e-8
        
    def forward(self,output,target):
        return torch.sqrt(F.mse_loss(output,target)+self.eps)

metrics = RMSELoss()
def loss_fn(pred, labels, metrics=metrics):
  return metrics(pred.view(-1), labels.view(-1))

In [18]:
def train_fold(n_fold):
  best_score = np.inf
  best_model = []
  best_pred = 0

  train_loader, valid_loader = get_dataloaders(df, n_fold)
  model = CLPmodel().to(DEVICE)
  optimizer = optim.Adam(params=model.parameters(), lr=5e-5)
  lr_scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=len(train_loader))
  metrics = RMSELoss()

  for epoch in range(EPOCHS):
    train_loss = 0
    for DATA in tqdm(train_loader):
      loss = train_fn(model, DATA, loss_fn, optimizer, lr_scheduler)
      train_loss += loss
    train_loss /= len(train_loader)

    valid_loss = 0
    valid_pred = np.ones(1)
    for DATA in tqdm(valid_loader):
      loss, pred = valid_fn(model, DATA, loss_fn, optimizer)
      valid_pred = np.concatenate([valid_pred, pred.reshape(-1,)], axis=0)
      valid_loss += loss
    valid_loss /= len(valid_loader)

    print(f"fold : {n_fold}, epoch : {epoch}, train_loss : {train_loss}")
    print(f"fold : {n_fold}, epoch : {epoch}, valid_loss : {valid_loss}")
    print('='*30)

    if valid_loss < best_score:
      best_socre = valid_loss
      del best_model, best_pred
      gc.collect()
      best_model = copy.deepcopy(model)
      best_pred  = valid_pred[1:]   
  return best_model, best_pred

In [19]:
EPOCHS = 5
DEVICE = 'cuda:0'

oof = np.zeros(df['target'].shape)
best_models = []

for n_fold in range(NUM_FOLDS):
  best_model, best_pred = train_fold(n_fold)
  best_models.append(best_model.cpu())
  oof[(df['kfold']==n_fold)] = best_pred
  print('-='*20)

print('%'*50)
oof_score = loss_fn(pred   = torch.tensor(oof).to(DEVICE),
                    labels = torch.tensor(df['target'].values).to(DEVICE) )
print('oof_score :', oof_score.cpu().detach().numpy())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



fold : 0, epoch : 0, train_loss : 0.7297217363928571
fold : 0, epoch : 0, valid_loss : 0.5638305719049883



fold : 0, epoch : 1, train_loss : 0.545834801696911
fold : 0, epoch : 1, valid_loss : 0.6010475966712119



fold : 0, epoch : 2, train_loss : 0.485487349035721
fold : 0, epoch : 2, valid_loss : 0.5285432374918125


...


fold : 4, epoch : 0, train_loss : 0.708832846550399
fold : 4, epoch : 0, valid_loss : 0.5222556614120242



fold : 4, epoch : 1, train_loss : 0.5567167576540399
fold : 4, epoch : 1, valid_loss : 0.6456437668120357



fold : 4, epoch : 2, train_loss : 0.47813425665126696
fold : 4, epoch : 2, valid_loss : 0.48297724647211354



fold : 4, epoch : 3, train_loss : 0.4299396433435525
fold : 4, epoch : 3, valid_loss : 0.5326125943954562



fold : 4, epoch : 4, train_loss : 0.6344243262810682
fold : 4, epoch : 4, valid_loss : 0.574197526863763
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
oof_score : 0.695861186284968
